In [42]:
using JuMP, Clp, NamedArrays

required_cars = [10, 6, 8, 11, 9, 7, 15, 7, 9, 12]
cars_present = [8, 13, 4, 8, 12, 2, 14, 11, 15, 7]
x_coord = [0, 20, 18, 30, 35, 33, 5, 5, 11, 2]
y_coord = [0, 20, 10, 12, 0, 25, 27, 10, 0, 15]
dist = zeros(10,10);
cost_per_mile = 0.5
agency = [:agency1, :agency2, :agency3, :agency4, :agency5, :agency6, :agency7, :agency8, :agency9, :agency10]

# calculate the distance from one location to the next
for i in 1:10
   for j in 1:10
      dist[i,j] = (1.3*sqrt( (x_coord[i] - x_coord[j])^2 + (y_coord[i] - y_coord[j])^2));
   end
end

#distance = NamedArray(dist, (agency,agency), ("agency", "agency"))

m = Model(solver=ClpSolver())

#@variable(m, hold[1:10] >= 0) # cars present after transportation
#@variable(m, cost[1:10] >= 0) # cost of each agency each night
#@variable(m, car_need[1:10]) # cars that each agency needs to get, positive means need

@variable(m, x[1:10,1:10] >= 0)          # x[i,j] is lumber shipped from site i to mill j.

@constraint(m, sup[i in 1:10], sum(x[i,j] for j in 1:10) == cars_present[i] )   # supply constraint
@constraint(m, dem[j in 1:10], sum(x[i,j] for i in 1:10) == required_cars[j] )   # demand constraint

@objective(m, Min, cost_per_mile*sum( x[i,j]*dist[i,j] for i in 1:10, j in 1:10 ) )  

status = solve(m)

#@constraint(m, flow[i in 1:10], cars_present[i]+car_need[i]==required_cars[i])

#@objective(m, Min, sum(cost));
println(status)
solution = NamedArray( Int[getvalue(x[i,j]) for i in 1:10, j in 1:10], (agency,agency), ("agency","agency") )
print(solution)
print(getobjectivevalue(m))

Optimal
[8 0 0 0 0 0 0 0 0 0; 0 6 1 0 0 5 1 0 0 0; 0 0 4 0 0 0 0 0 0 0; 0 0 0 8 0 0 0 0 0 0; 0 0 0 3 9 0 0 0 0 0; 0 0 0 0 0 2 0 0 0 0; 0 0 0 0 0 0 14 0 0 0; 0 0 0 0 0 0 0 6 0 5; 2 0 3 0 0 0 0 1 9 0; 0 0 0 0 0 0 0 0 0 7]152.63901632295628

In [39]:
test = [10, 6, 8, 11, 9, 7, 15, 7, 9, 12]
print(size(test))
test2 = [10 6 8 11 9 7 15 7 9 12]
print(size(test2))

(10,)(1, 10)

for i in 1:10
    while(required_cars[i] > cars_present[i])
        temp = dist[i,2]; # Find nearest agency
        car_out_agency = 0; # index of the agency send out cars
        for j in 2:10 # don't compare with itself
            if(dist[i,j] < temp)
                temp = dist[i,j];
                car_out_agency = j;
            end
        end
        if(car_need[car_out_agency] > car_need[i])
            cost[i] = dist[i,car_out_agency] * car_need[i] * 0.5;
            car_need[car_out_agency] = cars_present[j] + car_need[i];
            car_need[i] = cars_present[i] + car_need[car_out_agency];
        end
    end
    cost[i] = 0;
end